### **1. preprocessing에 필요한 library import.**

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import json

### **2. 수집한 기업 관련 Naver 뉴스 URL들을 news_info에 저장.**

- 각 url을 html 파일로 변환하여 news에 저장.
    
- BeautifulSoup를 활용하여 html 파일에서 뉴스의 제목, 작성 날짜, 기사 본문만을 추출.

- 기사 본문의 경우, 불필요한 문자, 테그, 사진 설명등을 제거하는 전처리 진행.

In [ ]:
urls = []
 
with open("news_url.txt", "r") as file:
    for line in file.readlines():
        url = line.rstrip()
        urls.append(url)
            
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
news_info = {}
index = 0

for url in urls:
    news_info[index] = {}
    
    news = requests.get(url, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")
    
    # 1. Naver 뉴스 기사 제목
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    title = re.sub(pattern='<[^>]*>', repl='', string=str(title))
    
    news_info[index]['title'] = title
    
    # 2. Naver 뉴스 기사 작성 날짜
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern='<[^>]*>', repl='', string=str(news_date))
    
    news_info[index]['date'] = news_date
    
    # 3. Naver 뉴스 기사 본문
    document = news_html.select("article#dic_area")
    if document == []:
        document = news_html.select("#articeBody") 
    document = ''.join(str(document))

    while True:
        start = document.find("<em")
        if start == -1:
            break
        i = 3
        while document[start+i] != '>':
            i += 1
        document = document.replace(document[start:start+i+1], "<")
    document = document.replace("</em>", ">")
    document = re.sub(pattern='<[^>]*>', repl='\n', string=document)
    document = document[1:-1].lstrip().rstrip()
    document_by_line = []
    for line in document.split('\n'):
        if line != "":
            document_by_line.append(line)
    document = '\n'.join(document_by_line)
            
    news_info[index]['document'] = document
    
    # 4. Naver 뉴스 기사 url
    news_info[index]['source'] = url
    index += 1

### **3. news_info를 news_info.json으로 저장.**

In [ ]:
with open("news_info.json", "w") as f:
    json.dump(news_info, f)